# Import required libraries

In [11]:
import math
import torch
import json
import numpy as np
import torch.utils.data
import torch.optim as optim
import torch.nn as nn
import random

from auxiliary import load_data, prepare_data_loaders
from algorithms import MLP

#Prepare data,model parameters and dataloaders

In [20]:
json_file = "data.json"
inp_rss_vals, gt_locations = load_data(json_file)  # load_data fonksiyonunu kullanarak veriyi yükle

batch_size = 32
train_test_split = 0.8
number_of_training_iters = 401

train_loader, test_loader, tensor_x_train, tensor_y_train, tensor_x_test, tensor_y_test = prepare_data_loaders(
    inp_rss_vals, gt_locations, batch_size, train_test_split)  # prepare_data_loaders fonksiyonunu kullanarak veriyi hazırla


# Train model

In [14]:
model = MLP() # MLP modelini oluştur
model.train() # modeli eğitim moduna al


criterion = nn.MSELoss(reduction='mean') # loss fonksiyonu olarak Mean Squared Error kullan
optimizer = optim.Adam(model.parameters(), lr=1e-3) # optimizer olarak Adam kullan


for i in range(number_of_training_iters): # number_of_training_iters kadar eğitim yap
    running_loss = 0.0 # başlangıçta loss değeri 0
    for inputs, labels in train_loader: # train_loader'dan her bir batch için
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if i+1 % 1 == 0: # her 20 epoch'ta
        print(f'Epoch [{i + 1}/{number_of_training_iters}] running accumulative loss across all batches: {running_loss:.3f}')



Epoch [1/401] running accumulative loss across all batches: 225.687


# Evaluate model performance

In [15]:
model.eval() # modeli test moduna al

predicted_locations_trainset = model(tensor_x_train) # eğitim seti için tahmin edilen konumları al
#print("Predicted locations on training set:", predicted_locations_trainset)


predicted_locations_test = model(tensor_x_test) # test seti için tahmin edilen konumları al
#print("Predicted locations on test set:", predicted_locations_test)

# Save the model parameters

In [19]:
torch.save(model.state_dict(), 'model_parameters.pth')